# Supervisor Architecture Prototype

This notebook is for rapid prototyping and testing of the new supervisor-based agent architecture for the Medicare Navigator.

## Architecture Overview

The new architecture replaces the single PatientNavigatorAgent with a **Supervisor Team**:

1. **Workflow Prescription Agent** - Identifies user intent and determines required workflows
2. **Document Requirement Agent** - Determines what documents are needed
3. **Workflow Router** - Routes to appropriate workflows based on prescription

## Workflows
- **Retrieval** - Information gathering and knowledge base search
- **Strategy** - Service access strategy and regulatory compliance  
- **Eligibility** - Eligibility parsing and application evaluation (placeholder)
- **Forms** - Form preparation and submission planning (placeholder)

Let's prototype and test each component!


In [ ]:
# Setup and Imports
import os
import sys
import asyncio
import json
from typing import Dict, Any, List, Optional, Tuple
from datetime import datetime
from dataclasses import dataclass
from enum import Enum

# Add project root to path
project_root = "/Users/aq_home/1Projects/accessa/insurance_navigator"
if project_root not in sys.path:
    sys.path.append(project_root)

# Core imports
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

print("✅ Basic imports successful")

# Try importing existing agents for reference
try:
    from agents.base_agent import BaseAgent
    from agents.common.exceptions import AgentException
    from utils.config_manager import ConfigManager
    print("✅ Core agent infrastructure imported")
except ImportError as e:
    print(f"⚠️ Could not import core agents: {e}")
    print("Will define minimal versions for prototyping")

# Check if we can access existing agents
try:
    from agents import TaskRequirementsAgent, ServiceAccessStrategyAgent
    print("✅ Existing specialized agents imported")
    EXISTING_AGENTS_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ Could not import existing agents: {e}")
    print("Will create mock versions for prototyping")
    EXISTING_AGENTS_AVAILABLE = False


## 1. Define Data Models

First, let's define the data models for our new supervisor agents and workflow system.


In [ ]:
# Data Models for New Architecture

class WorkflowType(Enum):
    """Available workflow types"""
    RETRIEVAL = "retrieval"
    STRATEGY = "strategy" 
    ELIGIBILITY = "eligibility"
    FORMS = "forms"

@dataclass
class UserContext:
    """User context information"""
    user_id: str
    conversation_id: str
    location: Optional[str] = None
    insurance: Optional[str] = None
    demographics: Optional[Dict[str, Any]] = None
    previous_interactions: Optional[List[Dict[str, Any]]] = None

class WorkflowPrescriptionResult(BaseModel):
    """Result from Workflow Prescription Agent"""
    workflows: List[str] = Field(description="List of required workflow names")
    confidence: float = Field(description="Confidence score for prescription")
    rationale: str = Field(description="Explanation of why these workflows were prescribed")
    priority_order: List[str] = Field(description="Suggested order of workflow execution")
    estimated_complexity: str = Field(description="Expected complexity: simple, moderate, complex")

class DocumentRequirementResult(BaseModel):
    """Result from Document Requirement Agent"""
    required_documents: List[str] = Field(description="List of required document types")
    sufficient_documents: bool = Field(description="Whether sufficient documents are available")
    missing_documents: List[str] = Field(description="List of missing document types")
    document_instructions: Dict[str, str] = Field(description="Instructions for obtaining missing documents")
    urgency_level: str = Field(description="Urgency level: low, medium, high")

class WorkflowExecutionState(BaseModel):
    """State tracking for workflow execution"""
    prescribed_workflows: List[str] = Field(default_factory=list)
    completed_workflows: List[str] = Field(default_factory=list)
    current_workflow: Optional[str] = None
    workflow_results: Dict[str, Any] = Field(default_factory=dict)
    has_sufficient_documents: bool = False
    requires_user_input: bool = False

# Workflow-specific result models
class RetrievalWorkflowResult(BaseModel):
    """Result from retrieval workflow"""
    retrieved_information: Dict[str, Any] = Field(default_factory=dict)
    sources: List[str] = Field(default_factory=list)
    confidence_score: float = 0.0
    information_gaps: List[str] = Field(default_factory=list)

class StrategyWorkflowResult(BaseModel):
    """Result from strategy workflow"""
    recommended_service: str = ""
    action_plan: List[Dict[str, Any]] = Field(default_factory=list)
    estimated_timeline: str = ""
    estimated_cost: Optional[str] = None
    network_providers: List[str] = Field(default_factory=list)
    regulatory_compliance: Dict[str, Any] = Field(default_factory=dict)

print("✅ Data models defined successfully")


## 2. Prototype Workflow Prescription Agent

This agent analyzes user messages and determines which workflows are needed.


In [ ]:
class WorkflowPrescriptionAgent:
    """Agent that determines which workflows are needed based on user intent"""
    
    def __init__(self, llm=None):
        self.llm = llm or ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0.1)
        self.output_parser = PydanticOutputParser(pydantic_object=WorkflowPrescriptionResult)
        
        # Define the prompt template
        self.prompt_template = PromptTemplate(
            template="""You are the Workflow Prescription Agent for a Medicare Navigator system.
            
Your job is to analyze user messages and determine which workflows are needed to fully address their request.

Available workflows:
- RETRIEVAL: Information gathering, knowledge base search, policy lookups
- STRATEGY: Service access strategy, provider network analysis, regulatory compliance
- ELIGIBILITY: Eligibility checking, application evaluation, benefit verification
- FORMS: Form preparation, submission planning, document assembly

User Context: {user_context}
User Message: {message}

Analyze the user's request and determine:
1. Which workflows are needed (can be multiple)
2. Your confidence in this prescription
3. The rationale for your choices
4. The suggested priority order
5. Expected complexity level

Examples:
- "I need to find a cardiologist" → RETRIEVAL + STRATEGY
- "Am I eligible for Medicare Part D?" → RETRIEVAL + ELIGIBILITY  
- "Help me apply for extra help with prescription costs" → RETRIEVAL + ELIGIBILITY + FORMS
- "What does Medicare cover for dental work?" → RETRIEVAL only

{format_instructions}""",
            input_variables=["message", "user_context"],
            partial_variables={"format_instructions": self.output_parser.get_format_instructions()}
        )
        
        self.chain = self.prompt_template | self.llm | self.output_parser
    
    async def prescribe_workflows(self, message: str, user_context: UserContext = None) -> WorkflowPrescriptionResult:
        """Analyze message and prescribe required workflows"""
        try:
            context_str = f"User ID: {user_context.user_id}, Location: {user_context.location}, Insurance: {user_context.insurance}" if user_context else "No context available"
            
            result = await self.chain.ainvoke({
                "message": message,
                "user_context": context_str
            })
            
            return result
            
        except Exception as e:
            print(f"Error in workflow prescription: {e}")
            # Return fallback result
            return WorkflowPrescriptionResult(
                workflows=["retrieval"],
                confidence=0.5,
                rationale=f"Error occurred, defaulting to retrieval workflow: {e}",
                priority_order=["retrieval"],
                estimated_complexity="simple"
            )
    
    def prescribe_workflows_sync(self, message: str, user_context: UserContext = None) -> WorkflowPrescriptionResult:
        """Synchronous version for testing"""
        try:
            context_str = f"User ID: {user_context.user_id}, Location: {user_context.location}, Insurance: {user_context.insurance}" if user_context else "No context available"
            
            result = self.chain.invoke({
                "message": message,
                "user_context": context_str
            })
            
            return result
            
        except Exception as e:
            print(f"Error in workflow prescription: {e}")
            return WorkflowPrescriptionResult(
                workflows=["retrieval"],
                confidence=0.5,
                rationale=f"Error occurred, defaulting to retrieval workflow: {e}",
                priority_order=["retrieval"],
                estimated_complexity="simple"
            )

# Test the Workflow Prescription Agent
print("🧪 Testing Workflow Prescription Agent...")

# Initialize agent
workflow_agent = WorkflowPrescriptionAgent()

# Test messages
test_messages = [
    "I need to find a cardiologist in my area",
    "Am I eligible for Medicare Part D?",
    "What does Medicare cover for dental work?",
    "Help me apply for extra help with prescription costs",
    "I want to change my Medicare plan",
    "Can you help me understand my coverage?"
]

print("\nTesting workflow prescriptions:")
for i, message in enumerate(test_messages, 1):
    print(f"\n{i}. Message: '{message}'")
    try:
        user_ctx = UserContext(user_id="test_user", conversation_id="test_conv", location="California", insurance="Medicare")
        result = workflow_agent.prescribe_workflows_sync(message, user_ctx)
        print(f"   Workflows: {result.workflows}")
        print(f"   Priority: {result.priority_order}")
        print(f"   Confidence: {result.confidence}")
        print(f"   Complexity: {result.estimated_complexity}")
        print(f"   Rationale: {result.rationale}")
    except Exception as e:
        print(f"   ❌ Error: {e}")

print("\n✅ Workflow Prescription Agent testing complete")


In [ ]:
class DocumentRequirementAgent:
    """Agent that determines document requirements for prescribed workflows"""
    
    def __init__(self, llm=None):
        self.llm = llm or ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0.1)
        self.output_parser = PydanticOutputParser(pydantic_object=DocumentRequirementResult)
        
        # Document requirements mapping
        self.workflow_document_map = {
            "retrieval": ["Medicare card", "identification"],
            "strategy": ["Medicare card", "medical records", "preferred provider list"],
            "eligibility": ["Medicare card", "tax returns", "Social Security statement", "medical records"],
            "forms": ["Medicare card", "tax returns", "bank statements", "medical records", "identification"]
        }
        
        self.prompt_template = PromptTemplate(
            template="""You are the Document Requirement Agent for a Medicare Navigator system.

Your job is to determine what documents are required based on:
1. The user's original message
2. The prescribed workflows that will be executed

User Message: {message}
Prescribed Workflows: {workflows}
User Context: {user_context}

Document Requirements by Workflow:
- RETRIEVAL: Medicare card, identification
- STRATEGY: Medicare card, medical records, preferred provider list, insurance cards
- ELIGIBILITY: Medicare card, tax returns, Social Security statement, medical records, proof of income
- FORMS: Medicare card, tax returns, bank statements, medical records, identification, proof of residence

Consider:
- What documents are absolutely required vs. helpful
- Whether the user likely has these documents readily available
- Instructions for obtaining missing documents
- Urgency level based on the user's request

Determine:
1. Required documents for the prescribed workflows
2. Whether sufficient documents are likely available
3. Missing documents that need to be obtained
4. Instructions for getting missing documents
5. Urgency level (low/medium/high)

{format_instructions}""",
            input_variables=["message", "workflows", "user_context"],
            partial_variables={"format_instructions": self.output_parser.get_format_instructions()}
        )
        
        self.chain = self.prompt_template | self.llm | self.output_parser
    
    async def analyze_requirements(self, message: str, prescribed_workflows: List[str], user_context: UserContext = None) -> DocumentRequirementResult:
        """Analyze document requirements for prescribed workflows"""
        try:
            context_str = f"User ID: {user_context.user_id}, Location: {user_context.location}, Insurance: {user_context.insurance}" if user_context else "No context available"
            
            result = await self.chain.ainvoke({
                "message": message,
                "workflows": ", ".join(prescribed_workflows),
                "user_context": context_str
            })
            
            return result
            
        except Exception as e:
            print(f"Error in document requirement analysis: {e}")
            # Return fallback based on workflow mapping
            all_docs = set()
            for workflow in prescribed_workflows:
                all_docs.update(self.workflow_document_map.get(workflow, []))
            
            return DocumentRequirementResult(
                required_documents=list(all_docs),
                sufficient_documents=False,
                missing_documents=list(all_docs),
                document_instructions={"error": f"Analysis failed: {e}"},
                urgency_level="medium"
            )
    
    def analyze_requirements_sync(self, message: str, prescribed_workflows: List[str], user_context: UserContext = None) -> DocumentRequirementResult:
        """Synchronous version for testing"""
        try:
            context_str = f"User ID: {user_context.user_id}, Location: {user_context.location}, Insurance: {user_context.insurance}" if user_context else "No context available"
            
            result = self.chain.invoke({
                "message": message,
                "workflows": ", ".join(prescribed_workflows),
                "user_context": context_str
            })
            
            return result
            
        except Exception as e:
            print(f"Error in document requirement analysis: {e}")
            # Return fallback based on workflow mapping
            all_docs = set()
            for workflow in prescribed_workflows:
                all_docs.update(self.workflow_document_map.get(workflow, []))
            
            return DocumentRequirementResult(
                required_documents=list(all_docs),
                sufficient_documents=False,
                missing_documents=list(all_docs),
                document_instructions={"error": f"Analysis failed: {e}"},
                urgency_level="medium"
            )

# Test the Document Requirement Agent
print("🧪 Testing Document Requirement Agent...")

# Initialize agent
doc_agent = DocumentRequirementAgent()

# Test cases with different workflow combinations
test_cases = [
    {
        "message": "I need to find a cardiologist in my area",
        "workflows": ["retrieval", "strategy"]
    },
    {
        "message": "Am I eligible for Medicare Part D?",
        "workflows": ["retrieval", "eligibility"]
    },
    {
        "message": "Help me apply for extra help with prescription costs",
        "workflows": ["retrieval", "eligibility", "forms"]
    },
    {
        "message": "What does Medicare cover for dental work?",
        "workflows": ["retrieval"]
    }
]

print("\nTesting document requirement analysis:")
for i, test_case in enumerate(test_cases, 1):
    print(f"\n{i}. Message: '{test_case['message']}'")
    print(f"   Workflows: {test_case['workflows']}")
    try:
        user_ctx = UserContext(user_id="test_user", conversation_id="test_conv", location="California", insurance="Medicare")
        result = doc_agent.analyze_requirements_sync(test_case["message"], test_case["workflows"], user_ctx)
        print(f"   Required documents: {result.required_documents}")
        print(f"   Sufficient documents: {result.sufficient_documents}")
        print(f"   Missing documents: {result.missing_documents}")
        print(f"   Urgency: {result.urgency_level}")
        if result.document_instructions:
            print(f"   Instructions: {list(result.document_instructions.keys())}")
    except Exception as e:
        print(f"   ❌ Error: {e}")

print("\n✅ Document Requirement Agent testing complete")


In [ ]:
# Mock Information Retrieval Agent
class InformationRetrievalAgent:
    """Handles information gathering and knowledge base search"""
    
    def __init__(self):
        self.knowledge_base = {
            "medicare_coverage": {
                "dental": "Medicare Part A and Part B don't cover most dental care, but Medicare Advantage plans may include dental coverage.",
                "vision": "Medicare Part A and Part B don't cover routine eye care, but Medicare Advantage plans may include vision coverage.",
                "prescription": "Medicare Part D covers prescription drugs. You can get Part D coverage through a standalone plan or Medicare Advantage plan."
            },
            "providers": {
                "cardiologist": "Cardiology services are covered under Medicare Part B when medically necessary.",
                "primary_care": "Primary care visits are covered under Medicare Part B with no deductible."
            }
        }
    
    async def retrieve_information(self, query: str, context: Dict[str, Any]) -> RetrievalWorkflowResult:
        """Retrieve information based on query"""
        
        # Mock retrieval logic
        retrieved_info = {}
        sources = []
        confidence = 0.8
        gaps = []
        
        query_lower = query.lower()
        
        # Search through knowledge base
        for category, items in self.knowledge_base.items():
            for key, value in items.items():
                if key in query_lower or any(word in query_lower for word in key.split()):
                    retrieved_info[f"{category}_{key}"] = value
                    sources.append(f"Medicare.gov - {category}")
        
        if not retrieved_info:
            retrieved_info["general"] = "I can help you navigate Medicare options and find healthcare services."
            gaps.append("Specific information not found in knowledge base")
            confidence = 0.4
        
        return RetrievalWorkflowResult(
            retrieved_information=retrieved_info,
            sources=sources,
            confidence_score=confidence,
            information_gaps=gaps
        )

# Mock Strategy Workflow Agent  
class StrategyWorkflowAgent:
    """Handles service access strategy and provider network analysis"""
    
    def __init__(self):
        self.provider_networks = {
            "California": {
                "cardiologists": ["Dr. Smith - Cedars-Sinai", "Dr. Johnson - UCLA Medical"],
                "primary_care": ["Dr. Williams - Kaiser Permanente", "Dr. Brown - Sutter Health"]
            }
        }
    
    async def develop_strategy(self, query: str, context: Dict[str, Any], retrieved_info: Dict[str, Any]) -> StrategyWorkflowResult:
        """Develop access strategy based on query and retrieved information"""
        
        location = context.get("location", "Unknown")
        query_lower = query.lower()
        
        # Determine service type
        service_type = "general_healthcare"
        if "cardiologist" in query_lower:
            service_type = "cardiology"
        elif "primary" in query_lower or "doctor" in query_lower:
            service_type = "primary_care"
        
        # Get providers for location
        providers = []
        if location in self.provider_networks:
            if service_type == "cardiology":
                providers = self.provider_networks[location].get("cardiologists", [])
            elif service_type == "primary_care":
                providers = self.provider_networks[location].get("primary_care", [])
        
        # Create action plan
        action_plan = [
            {
                "step_number": 1,
                "step_description": f"Verify your Medicare coverage for {service_type} services",
                "expected_timeline": "1-2 days"
            },
            {
                "step_number": 2,
                "step_description": f"Contact providers in your area to check availability",
                "expected_timeline": "3-5 days"
            },
            {
                "step_number": 3,
                "step_description": "Schedule your appointment",
                "expected_timeline": "1-2 weeks"
            }
        ]
        
        return StrategyWorkflowResult(
            recommended_service=service_type,
            action_plan=action_plan,
            estimated_timeline="2-3 weeks",
            estimated_cost="Varies by service",
            network_providers=providers,
            regulatory_compliance={"medicare_approved": True, "prior_auth_required": False}
        )

# Placeholder Eligibility Agent
class EligibilityPlaceholderAgent:
    """Placeholder for eligibility workflow"""
    
    async def check_eligibility(self, query: str, context: Dict[str, Any]) -> Dict[str, Any]:
        """Placeholder eligibility check"""
        return {
            "status": "placeholder",
            "message": "Eligibility checking functionality will be implemented in future release",
            "confidence": 0.0
        }

# Placeholder Forms Agent  
class FormsPlaceholderAgent:
    """Placeholder for forms workflow"""
    
    async def prepare_forms(self, query: str, context: Dict[str, Any]) -> Dict[str, Any]:
        """Placeholder form preparation"""
        return {
            "status": "placeholder", 
            "message": "Form preparation functionality will be implemented in future release",
            "forms": []
        }

print("✅ Individual workflow agents defined")
